In [ ]:
include("../Algebra/Gradients.jl")
include("../Algebra/Hamiltonians.jl")
include("../Algebra/Matrices.jl")
include("../Algebra/Propagators.jl")
include("../Amplitudes/Chebyshev.jl")
include("../Costs/Costs.jl")
include("../Gates/Xgate.jl")

In [ ]:
using LinearAlgebra, Optim, Measures, CSV, DataFrames,Plots

In [ ]:
ω_c = 5000
ω_q = 3000
χ = 3e-1
N = 6
amp_c = 1
amp_q = 1
T = 0.5

### Target gate

In [ ]:
I_qubit = Matrix(I,2,2)
I_cavity = Matrix(I,N,N)
#X_gate = kron(x_gate(N, Array[[3,4], [5,6], [7,8]]), I_qubit)
#X_gate = kron(x_gate(N, Array[[1,2]]), I_qubit)
X_gate = kron(x_gate(N, Array[[3,4]]), I_qubit)

θ = π/2.5
RZ = kron(Rz(N, Array[[1,3]], θ), I_qubit)
RX = kron(Rx(N, Array[[3,4]], θ/2), I_qubit)

sx = kron(I_cavity, [0 1; 1 0])
sy = kron(I_cavity, [0 -1im; 1im 0])
# generating matrices
a,adag,sp,sm,sz = generate_matrices(N)

ψ_initial = zeros(N)
ψ_initial[1] = 1
ψ_initial = kron(ψ_initial/norm(ψ_initial), [0,1])

interaction_transformation(t) = cis(- (ω_c * a' * a + ω_q / 2 * sz) * t)
int_transformation = interaction_transformation(T)

In [ ]:
H_drift = χ * adag * a * sz / 2 #+ ω_c * adag * a + ω_q * sz / 2

In [ ]:
function cost_from_0_dispersive(H_drift, sp, sm, a, adag, T, δt, coefficients, unitary, ω_c, ω_q, initial_state, final_state, amp_q=1e-1, amp_c=1e-1)

    # initialising the propagator
    dim = size(H_drift,1)
    propagator = Matrix{ComplexF64}(I,dim,dim)
    amplitude_c(t) = chebyshev_amplitude(coefficients[1:Int(length(coefficients)/2)], T, t)
    amplitude_q(t) = chebyshev_amplitude(coefficients[Int(length(coefficients)/2) + 1:end], T, t)

    # time ordered product of the single exponential matrices
    # is this true at all? or we need more time steps aniway?
    for l in 0:δt:T
        H = H_drift + amp_q * (amplitude_q(l) * sp + amplitude_q(l)' * sm) + amp_c * (amplitude_c(l) * a +  amplitude_c(l)' * adag)
        infinitesimal_propagator  = cis(- H * δt)
        propagator = infinitesimal_propagator * propagator
    end

    c = final_state' * propagator * initial_state
   
    return 1 - norm(c)^2
end

function cost_from_0_dispersive_gate(H_drift, sp, sm, a, adag, T, δt, coefficients, unitary, ω_c, ω_q, amp_q, amp_c)

    # initialising the propagator
    dim = size(H_drift,1)
    propagator = Matrix{ComplexF64}(I,dim,dim)
    amplitude_c(t) = chebyshev_amplitude(coefficients[1:Int(length(coefficients)/2)], T, t)
    amplitude_q(t) = chebyshev_amplitude(coefficients[Int(length(coefficients)/2) + 1:end], T, t)

    # time ordered product of the single exponential matrices
    # is this true at all? or we need more time steps aniway?
    for l in 0:δt:T
        H = H_drift + amp_q * (amplitude_q(l) * sp + amplitude_q(l)' * sm) + amp_c * (amplitude_c(l) * a +  amplitude_c(l)' * adag)
        infinitesimal_propagator  = cis(- H * δt)
        propagator = infinitesimal_propagator * propagator
    end

    c = tr(unitary' * int_transformation * propagator)/dim
   
    return 1 - norm(c)^2
end

function cost_from_0_dispersive_gate_non_interaction(H_drift, sp, sm, a, adag, T, δt, coefficients, unitary, ω_c, ω_q, amp_q, amp_c)

    # initialising the propagator
    dim = size(H_drift,1)
    propagator = Matrix{ComplexF64}(I,dim,dim)
    amplitude_c(t) = chebyshev_amplitude(coefficients[1:Int(length(coefficients)/2)], T, t)
    amplitude_q(t) = chebyshev_amplitude(coefficients[Int(length(coefficients)/2) + 1:end], T, t)

    # time ordered product of the single exponential matrices
    # is this true at all? or we need more time steps aniway?
    for l in 0:δt:T
        H = H_drift + amp_q * (amplitude_q(l) * sp + amplitude_q(l)' * sm) + amp_c * (amplitude_c(l) * a +  amplitude_c(l)' * adag)
        infinitesimal_propagator  = cis(- H * δt)
        propagator = infinitesimal_propagator * propagator
    end

    c = tr(unitary' * propagator)/dim
   
    return 1 - norm(c)^2
end

function propagator(H_drift, sp, sm, a, adag, T, δt, coefficients)

    # initialising the propagator
    dim = size(H_drift,1)
    propagator = Matrix{ComplexF64}(I,dim,dim)

    amplitude_c = [ChebyshevT(coefficients[1:Int(length(coefficients)/2)])((2t - T)/T) for t in 0:δt:T]
    amplitude_q = [ChebyshevT(coefficients[Int(length(coefficients)/2) + 1:end])((2t - T)/T) for t in 0:δt:T]


    # time ordered product of the single exponential matrices
    # is this true at all? or we need more time steps aniway?
    for n in 1:1:Int(T/δt)+1
        H = H_drift + (amplitude_q(n) * sp + amplitude_q(n)' * sm) + (amplitude_c(n) * a + amplitude_c(n)' * adag)
        infinitesimal_propagator  = cis(- H * δt)
        propagator = infinitesimal_propagator * propagator
    end

    return propagator * exp(-1im*angle(propagator[8, 8]))
    
end

function gradient(coefficients, unitary, N, h, cost_before_increment,cost)

    L = length(coefficients)
    gradient = zeros(L)

    for i in 1:L
        coeffs_copy = copy(coefficients)
        coeffs_copy[i] = coefficients[i] + h
        gradient[i] = (cost(coeffs_copy) - cost_before_increment) / h
    end

    return gradient

end

In [ ]:
snap(x) = kron(Diagonal(exp.(-1im.*x)),I_qubit)
snap_params = rand(N) * 2 * pi .- pi
SNAP_GATE = snap(snap_params)

In [ ]:
f(x) = cost_from_0_dispersive(H_drift, sp, sm, a, adag, T, T / (10 * 3) , x, X_gate, ω_c, ω_q, ψ_initial, ψ_final_x)
g(x) = cost_from_0_dispersive_gate_non_interaction(H_drift, sp, sm, a, adag, T, T/50, x, X_gate, ω_c, ω_q, 1, 1)
r(x) = cost_from_0_dispersive_gate(H_drift, sp, sm, a, adag, T, T/50, x, RX, ω_c, ω_q, 1, 1)
s(x) = cost_from_0_dispersive_gate(H_drift, sp, sm, a, adag, T, T/50, x, SNAP_GATE, ω_c, ω_q, 1, 1)

In [ ]:
for i in 1:1
    println("iteration number: ", i )
    rand_coeffs_20 = rand(16)*2 .-1 + (rand(16)*2im .-1im)
    res_LBFGS = Optim.optimize(g, rand_coeffs_20 , LBFGS(), Optim.Options(show_trace=true, show_every=5, iterations=100))
    println(Optim.minimizer(res_LBFGS))
    println(Optim.minimum(res_LBFGS))
    rand_coeffs_20 = zeros(4) + (zeros(4)*im)
    res_LBFGS = Optim.optimize(g, [Optim.minimizer(res_LBFGS)[1:8]...,rand_coeffs_20...,Optim.minimizer(res_LBFGS)[9:end]...,rand_coeffs_20...] , LBFGS(), Optim.Options(show_trace=true, show_every=5,iterations=200))
    println(Optim.minimizer(res_LBFGS))
    println(Optim.minimum(res_LBFGS))
    res_LBFGS = Optim.optimize(g, [Optim.minimizer(res_LBFGS)[1:12]...,rand_coeffs_20...,Optim.minimizer(res_LBFGS)[13:end]...,rand_coeffs_20...] , LBFGS(), Optim.Options(show_trace=true, show_every=5,iterations=200))
    println(Optim.minimizer(res_LBFGS))
    println(Optim.minimum(res_LBFGS))
    res_LBFGS = Optim.optimize(g, [Optim.minimizer(res_LBFGS)[1:16]...,rand_coeffs_20...,Optim.minimizer(res_LBFGS)[17:end]...,rand_coeffs_20...] , LBFGS(), Optim.Options(show_trace=true, show_every=5,iterations=200))
    println(Optim.minimizer(res_LBFGS))
    println(Optim.minimum(res_LBFGS))
    res_LBFGS = Optim.optimize(g, [Optim.minimizer(res_LBFGS)[1:20]...,rand_coeffs_20...,Optim.minimizer(res_LBFGS)[21:end]...,rand_coeffs_20...] , LBFGS(), Optim.Options(show_trace=true, show_every=5,iterations=300))
    println(Optim.minimizer(res_LBFGS))
    println(Optim.minimum(res_LBFGS))
end

In [ ]:
snap_params = rand(N) * 2 * pi .- pi
SNAP_GATE = snap(snap_params)
x(x) = cost_from_0_dispersive_gate(H_drift, sp, sm, a, adag, T, T/50, x, SNAP_GATE, ω_c, ω_q, 10, 10)
println(rand(N) * 2 * pi .- pi)

it_costs_x = []
coefficients_x = []
it_costs_snap = []
coefficients_snap = []
it_costs_rx = []
coefficients_rx = []
for iteration in 1:15
    coefficients = rand(48)*2 .-1 + (rand(48)*2im .-1im)
    println(" iterations :", iteration)
    res_x = Optim.optimize(x, coefficients , LBFGS())
    # res_rx = Optim.optimize(r, coefficients , LBFGS(), Optim.Options(iterations=1000))
    # res_snap = Optim.optimize(s, coefficients , LBFGS(), Optim.Options(iterations=1000))
    push!(it_costs_x,Optim.minimum(res_x))
    # push!(it_costs_rx,Optim.minimum(res_rx))
    # push!(it_costs_snap,Optim.minimum(res_snap))
    push!(coefficients_x,Optim.minimizer(res_x))
    # push!(coefficients_rx,Optim.minimizer(res_rx))
    # push!(coefficients_snap,Optim.minimizer(res_snap))
end

In [ ]:
df_x = DataFrame(fidelity=it_costs_x, coefficients=coefficients_x)
# df_rx = DataFrame(fidelity=it_costs_rx, coefficients=coefficients_rx)
# df_snap = DataFrame(fidelity=it_costs_snap, coefficients=coefficients_snap)
CSV.write("../../data/snap_gate_100ns_50steps_1.txt", df_x)
# CSV.write("../../data/rx_gate_100ns_50steps.txt", df_rx)
# CSV.write("../../data/snap_gate_100ns_50steps.txt", df_snap)

In [ ]:
snap_params = rand(N) * 2 * pi .- pi
SNAP_GATE = snap(snap_params)
x(x) = cost_from_0_dispersive_gate(H_drift, sp, sm, a, adag, T, T/50, x, SNAP_GATE, ω_c, ω_q, 10, 10)
println(rand(N) * 2 * pi .- pi)

it_costs_x = []
coefficients_x = []
it_costs_snap = []
coefficients_snap = []
it_costs_rx = []
coefficients_rx = []
for iteration in 1:15
    coefficients = rand(48)*2 .-1 + (rand(48)*2im .-1im)
    println(" iterations :", iteration)
    res_x = Optim.optimize(x, coefficients , LBFGS())
    # res_rx = Optim.optimize(r, coefficients , LBFGS(), Optim.Options(iterations=1000))
    # res_snap = Optim.optimize(s, coefficients , LBFGS(), Optim.Options(iterations=1000))
    push!(it_costs_x,Optim.minimum(res_x))
    # push!(it_costs_rx,Optim.minimum(res_rx))
    # push!(it_costs_snap,Optim.minimum(res_snap))
    push!(coefficients_x,Optim.minimizer(res_x))
    # push!(coefficients_rx,Optim.minimizer(res_rx))
    # push!(coefficients_snap,Optim.minimizer(res_snap))
end

df_x = DataFrame(fidelity=it_costs_x, coefficients=coefficients_x)
# df_rx = DataFrame(fidelity=it_costs_rx, coefficients=coefficients_rx)
# df_snap = DataFrame(fidelity=it_costs_snap, coefficients=coefficients_snap)
CSV.write("../../data/snap_gate_100ns_50steps_2.txt", df_x)
# CSV.write("../../data/rx_gate_100ns_50steps.txt", df_rx)
# CSV.write("../../data/snap_gate_100ns_50steps.txt", df_snap)

In [ ]:
optimised_coeffs = coefficients_x[1]
println(optimised_coeffs)

In [ ]:
coeffs_cavity = optimised_coeffs[1:25]
coeffs_qubit = optimised_coeffs[26:end]

In [ ]:
d=1
amplitude_c(t) = chebyshev_amplitude(coeffs_cavity, T*d, t)
amplitude_q(t) = chebyshev_amplitude(coeffs_qubit, T*d, t)

amps_c = [amplitude_c(l)*10/d for l in 0:T*d/(50):T*d]
amps_q = [amplitude_q(l)*10/d for l in 0:T*d/(50):T*d]

In [ ]:
# plt2to6 = plot(1:1:8, costs_2to6, label="2 -> 6", linestyle=:dash, marker = :circle, yaxis=:log, legend=:bottomleft, xlim=(0.5,8.5), xlabel="Circuit depth", ylabel="Pulse [GHz]", yticks=exp10.(range(-16, stop=0, length=17)))
# plt0to4 = plot!(1:1:8, costs_0to4, label="0 -> 4", linestyle=:dash, marker = :circle, )
# pltSuperposition = plot!(1:1:8, costs_superposition, label="0 -> superposition", linestyle=:dash, marker = :circle )
# plt0to7 = plot!(1:1:8, costs, label="0 -> 7", linestyle=:dash, marker = :circle, fmt = :jpeg)


In [ ]:
imaginary_part = plot([l for l in 0:T*d /50:T*d], imag(amps_c), label = "Im[ϵ(t)]", linestyle=:dash, marker = :circle, markersize=3, xlabel="Time[μs]", ylabel="Pulse [MHz]", fmt = :PDF, layout=2, subplot=1, size=(1500,300), margin=10mm,xtickfontsize=14,ytickfontsize=14,xguidefontsize=14,yguidefontsize=14,legendfontsize=14)
real_part = plot!([l for l in 0:T*d /50:T*d], real(amps_c), label= "Re[ϵ(t)]", linestyle=:dash, marker = :circle, markersize=3, xlabel="Time[μs]", ylabel="Pulse [MHz]", fmt = :PDF, subplot=1)
imaginary_part = plot!([l for l in 0:T*d /50:T*d], imag(amps_q), label = "Im[Ω(t)]", linestyle=:dash, marker = :circle, markersize=3, xlabel="Time[μs]", ylabel="Pulse [MHz]", fmt = :PDF, subplot=2, xtickfontsize=14,ytickfontsize=14,xguidefontsize=14,yguidefontsize=14,legendfontsize=14)
real_part = plot!([l for l in 0:T*d /50:T*d], real(amps_q), label= "Re[Ω(t)]", linestyle=:dash, marker = :circle, markersize=3, xlabel="Time[μs]", ylabel="Pulse [MHz]", fmt = :PDF, subplot=2)
#savefig("../../plots/pulse_optimisation/x7photons_100timesteps_100ns_1dote-2Infidelity.pdf")

In [ ]:
g(x) = cost_from_0_dispersive_gate(H_drift, sp, sm, a, adag, T, T / (50), x, RZ, ω_c, ω_q, 1, 1)

In [ ]:
res_reseding = Optim.optimize(g,optimised_coeffs[1:18] , LBFGS(), Optim.Options(show_trace=true, show_every=50))

In [ ]:
optimised_coeffs_2 = Optim.minimizer(res_reseding)

In [ ]:
coeffs_cavity = optimised_coeffs_2[1:9]
coeffs_qubit = optimised_coeffs_2[10:end]

In [ ]:
amplitude_c(t) = chebyshev_amplitude(coeffs_cavity, T, t)
amplitude_q(t) = chebyshev_amplitude(coeffs_qubit, T, t)

amps_c = [amplitude_c(l) for l in 0:T/50:T]
amps_q = [amplitude_q(l)  for l in 0:T/50:T]

In [ ]:
imaginary_part = plot([l for l in 0:T /50:T], imag(amps_c), label = "Im[ϵ(t)]", linestyle=:dash, marker = :circle, markersize=3, xlabel="Time[ns]", ylabel="Pulse [GHz]", fmt = :PDF, layout=2, subplot=1, size=(800,300), margin=5mm)
real_part = plot!([l for l in 0:T /50:T], real(amps_c), label= "Re[ϵ(t)]", linestyle=:dash, marker = :circle, markersize=3, xlabel="Time[ns]", ylabel="Pulse [GHz]", fmt = :PDF, subplot=1)
imaginary_part = plot!([l for l in 0:T /50:T], imag(amps_q), label = "Im[Ω(t)]", linestyle=:dash, marker = :circle, markersize=3, xlabel="Time[ns]", ylabel="Pulse [GHz]", fmt = :PDF, subplot=2)
real_part = plot!([l for l in 0:T /50:T], real(amps_q), label= "Re[Ω(t)]", linestyle=:dash, marker = :circle, markersize=3, xlabel="Time[ns]", ylabel="Pulse [GHz]", fmt = :PDF, subplot=2)


In [ ]:
g(x) = cost_from_0_dispersive_gate(H_drift, sp, sm, a, adag, T*2, T*2 / (200), x, RZ, ω_c, ω_q, 1, 1)

In [ ]:
res_reseding_2 = Optim.optimize(g,optimised_coeffs_2 , LBFGS(), Optim.Options(show_trace=true, show_every=10))

In [ ]:
optimised_coeffs_3 = Optim.minimizer(res_reseding_2)
res_reseding_2_bis = Optim.optimize(g,optimised_coeffs_3 , LBFGS(), Optim.Options(show_trace=true, show_every=10))

In [ ]:
coeffs_cavity = optimised_coeffs_3[1:7]
coeffs_qubit = optimised_coeffs_3[8:end]
amplitude_c(t) = chebyshev_amplitude(coeffs_cavity, T*2, t)
amplitude_q(t) = chebyshev_amplitude(coeffs_qubit, T*2, t)

amps_c = [amplitude_c(l) for l in 0:T*2/(200):T*2]
amps_q = [amplitude_q(l)  for l in 0:T*2/(200):T*2]

In [ ]:
imaginary_part = plot([l for l in 0:T*2 / (200):T*2], imag(amps_c), label = "Im[ϵ(t)]", linestyle=:dash, marker = :circle, markersize=3, xlabel="Time[ns]", ylabel="Pulse [GHz]", fmt = :PDF, layout=2, subplot=1, size=(800,300), margin=5mm)
real_part = plot!([l for l in 0:T*2 / (200):T*2], real(amps_c), label= "Re[ϵ(t)]", linestyle=:dash, marker = :circle, markersize=3, xlabel="Time[ns]", ylabel="Pulse [GHz]", fmt = :PDF, subplot=1)
imaginary_part = plot!([l for l in 0:T*2 / (200):T*2], imag(amps_q), label = "Im[Ω(t)]", linestyle=:dash, marker = :circle, markersize=3, xlabel="Time[ns]", ylabel="Pulse [GHz]", fmt = :PDF, subplot=2)
real_part = plot!([l for l in 0:T*2 / (200):T*2], real(amps_q), label= "Re[Ω(t)]", linestyle=:dash, marker = :circle, markersize=3, xlabel="Time[ns]", ylabel="Pulse [GHz]", fmt = :PDF, subplot=2)

In [ ]:
optimised_coeffs_3_bis = Optim.minimizer(res_reseding_2_bis)
coeffs_cavity = optimised_coeffs_3_bis[1:7]
coeffs_qubit = optimised_coeffs_3_bis[8:end]
amplitude_c(t) = chebyshev_amplitude(coeffs_cavity, T*2, t)
amplitude_q(t) = chebyshev_amplitude(coeffs_qubit, T*2, t)

amps_c = [amplitude_c(l) for l in 0:T*2/(200):T*2]
amps_q = [amplitude_q(l)  for l in 0:T*2/(200):T*2]

imaginary_part = plot([l for l in 0:T*2 / (200):T*2], imag(amps_c), label = "Im[ϵ(t)]", linestyle=:dash, marker = :circle, markersize=3, xlabel="Time[ns]", ylabel="Pulse [GHz]", fmt = :PDF, layout=2, subplot=1, size=(800,300), margin=5mm)
real_part = plot!([l for l in 0:T*2 / (200):T*2], real(amps_c), label= "Re[ϵ(t)]", linestyle=:dash, marker = :circle, markersize=3, xlabel="Time[ns]", ylabel="Pulse [GHz]", fmt = :PDF, subplot=1)
imaginary_part = plot!([l for l in 0:T*2 / (200):T*2], imag(amps_q), label = "Im[Ω(t)]", linestyle=:dash, marker = :circle, markersize=3, xlabel="Time[ns]", ylabel="Pulse [GHz]", fmt = :PDF, subplot=2)
real_part = plot!([l for l in 0:T*2 / (200):T*2], real(amps_q), label= "Re[Ω(t)]", linestyle=:dash, marker = :circle, markersize=3, xlabel="Time[ns]", ylabel="Pulse [GHz]", fmt = :PDF, subplot=2)

In [ ]:
optimised_coeffs_3_bis = Optim.minimizer(res_reseding_2_bis)